In [1]:
def get_widget_or_default(widget_name: str, default_value: str) -> str:
    try:
        return dbutils.widgets.get(widget_name)
    except Exception:
        return default_value

gold_run_id = get_widget_or_default("gold_run_id", "MANUAL_RUN")

SILVER_TABLE = "silver_transactions"
GOLD_TABLE   = "gold_fact_transactions"

DIM_USER = "gold_dim_user"
DIM_CARD = "gold_dim_card"
DIM_MCC  = "gold_dim_mcc"
DIM_DATE = "gold_dim_date"

# Anomalies (recommandées)
ANOM_ORPHAN_USER = "gold_anom_fact_txn_orphan_user"
ANOM_ORPHAN_CARD = "gold_anom_fact_txn_orphan_card"
ANOM_ORPHAN_MCC  = "gold_anom_fact_txn_orphan_mcc"
ANOM_ORPHAN_DATE = "gold_anom_fact_txn_orphan_date"

print(f"gold_run_id = {gold_run_id}")
print(f"SILVER_TABLE = {SILVER_TABLE}")
print(f"GOLD_TABLE   = {GOLD_TABLE}")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 3, Finished, Available, Finished)

gold_run_id = MANUAL_RUN
SILVER_TABLE = silver_transactions
GOLD_TABLE   = gold_fact_transactions


In [2]:
#---------------------------------------
# Silver Contract first
#---------------------------------------

from pyspark.sql import functions as F

CONTRACT = {
  "natural_key": ["transaction_id"],
  "business_columns": [
    ("transaction_id","BIGINT",False),
    ("txn_ts","TIMESTAMP",False),
    ("txn_date","DATE",False),
    ("txn_month","DATE",False),
    ("client_id","BIGINT",False),
    ("card_id","BIGINT",False),
    ("merchant_id","BIGINT",False),
    ("mcc_code","STRING",False),
    ("amount","DECIMAL(18,2)",False),
    ("use_chip","STRING",True),
    ("merchant_city","STRING",True),
    ("merchant_state","STRING",True),
    ("zip","STRING",True),
    ("error_code","INT",False),
    ("is_success","BOOLEAN",False),
  ],
  "technical_columns": [
    ("source_file","STRING",True),
    ("ingestion_date","DATE",False),
    ("ingestion_ts","TIMESTAMP",False),
    ("record_hash","STRING",False),
  ],
  "allow_additional_columns": False,
  "dedup": {
    "keys": ["transaction_id"],
    "order_by": "ingestion_ts",
    "strategy": "keep_latest"
  }
}

BUSINESS_COLS = [c[0] for c in CONTRACT["business_columns"]]
TECH_COLS     = [c[0] for c in CONTRACT["technical_columns"]]
EXPECTED_COLS = BUSINESS_COLS + TECH_COLS


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 4, Finished, Available, Finished)

In [3]:
#---------------------------------------
# Lecture Silver + fail fast
#---------------------------------------

df_silver = spark.table(SILVER_TABLE)

actual_cols = df_silver.columns
missing = sorted(list(set(EXPECTED_COLS) - set(actual_cols)))
extra   = sorted(list(set(actual_cols) - set(EXPECTED_COLS)))

if missing:
    raise ValueError(f"[FAIL FAST] Missing columns in {SILVER_TABLE}: {missing}")

if not CONTRACT["allow_additional_columns"] and extra:
    raise ValueError(f"[FAIL FAST] Additional columns not allowed in {SILVER_TABLE}: {extra}")

print(f"Schema validation OK for {SILVER_TABLE}. Column count = {len(actual_cols)}")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 5, Finished, Available, Finished)

Schema validation OK for silver_transactions. Column count = 19


In [4]:
#---------------------------------------
# Projection Canonique + cast contractuel
#---------------------------------------

def cast_expr(col_name: str, spark_type: str):
    return F.col(col_name).cast(spark_type).alias(col_name)

select_exprs = []
for name, typ, _nullable in CONTRACT["business_columns"]:
    select_exprs.append(cast_expr(name, typ))

for name, typ, _nullable in CONTRACT["technical_columns"]:
    select_exprs.append(cast_expr(name, typ))

df_proj = df_silver.select(*select_exprs)


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 6, Finished, Available, Finished)

In [5]:
#---------------------------------------
# Deduplication
#---------------------------------------

from pyspark.sql.window import Window

w = Window.partitionBy(*CONTRACT["dedup"]["keys"]).orderBy(F.col(CONTRACT["dedup"]["order_by"]).desc())

df_dedup = (
    df_proj
    .withColumn("_rn", F.row_number().over(w))
    .filter(F.col("_rn") == 1)
    .drop("_rn")
)

dup_cnt = (
    df_dedup
    .groupBy("transaction_id")
    .count()
    .filter(F.col("count") > 1)
    .count()
)

if dup_cnt != 0:
    raise ValueError(f"[FAIL FAST] Deduplication failed: still have duplicates on transaction_id (count={dup_cnt})")

print(f"Dedup OK. Rows = {df_dedup.count()}")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 7, Finished, Available, Finished)

Dedup OK. Rows = 13305915


In [6]:
#---------------------------------------
# Referential dimensions load
#---------------------------------------

df_users = spark.table(DIM_USER).select(F.col("client_id").cast("BIGINT").alias("client_id")).dropDuplicates()
df_cards = spark.table(DIM_CARD).select(F.col("card_id").cast("BIGINT").alias("card_id")).dropDuplicates()
df_mcc   = spark.table(DIM_MCC).select(F.col("mcc_code").cast("STRING").alias("mcc_code")).dropDuplicates()
df_dates = spark.table(DIM_DATE).select(F.col("date_value").cast("DATE").alias("txn_date")).dropDuplicates()


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 8, Finished, Available, Finished)

In [7]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

ANOM_TABLE = "gold_anomaly_event"
ANOM_DOMAIN = "GOLD_FACT_TRANSACTIONS"

def append_anomalies(
    df: DataFrame,
    anom_type: str,
    severity: str,
    rule_id: str,
    details_json: str | None = None
) -> None:
    if df is None or df.rdd.isEmpty():
        return

    df_out = (
        df
        # anomaly identity
        .withColumn("anomaly_id", F.expr("uuid()"))
        .withColumn("anom_domain", F.lit(ANOM_DOMAIN))
        .withColumn("anom_type", F.lit(anom_type))
        .withColumn("severity", F.lit(severity))
        .withColumn("rule_id", F.lit(rule_id))

        # optional metric fields (kept null for row-level anomalies)
        .withColumn("metric_name", F.lit(None).cast("string"))
        .withColumn("metric_value", F.lit(None).cast("double"))

        # keep details compact
        .withColumn("details_json", F.lit(details_json).cast("string"))

        # gold audit
        .withColumn("gold_run_id", F.lit(gold_run_id))
        .withColumn("gold_load_ts", F.current_timestamp())
    )

    # Project only columns existing in anomaly table schema
    cols = [
        "anomaly_id","anom_domain","anom_type","severity","rule_id",
        "transaction_id","client_id","card_id","mcc_code","txn_ts","txn_date","txn_month",
        "metric_name","metric_value","details_json",
        "source_file","ingestion_date","ingestion_ts",
        "gold_run_id","gold_load_ts"
    ]
    df_out.select(*cols).write.mode("append").format("delta").saveAsTable(ANOM_TABLE)

    print(f"Anomalies appended to {ANOM_TABLE}: {anom_type} ({severity})")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 9, Finished, Available, Finished)

In [8]:
#---------------------------------------
# Conformance checks
#---------------------------------------

base_cols = [
    "transaction_id","client_id","card_id","mcc_code",
    "txn_ts","txn_date","txn_month",
    "source_file","ingestion_date","ingestion_ts"
]

df_orphan_user = (
    df_dedup.join(df_users, on="client_id", how="left_anti")
    .select(*base_cols)
)

df_orphan_card = (
    df_dedup.join(df_cards, on="card_id", how="left_anti")
    .select(*base_cols)
)

df_orphan_mcc = (
    df_dedup.join(df_mcc, on="mcc_code", how="left_anti")
    .select(*base_cols)
)

df_orphan_date = (
    df_dedup.join(df_dates, on="txn_date", how="left_anti")
    .select(*base_cols)
)



StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 10, Finished, Available, Finished)

In [9]:
#---------------------------------------
# Persist Anomaly events
#---------------------------------------

append_anomalies(
    df_orphan_user,
    anom_type="ORPHAN_DIM_USER",
    severity="ERROR",
    rule_id="GOLD_TXN_FK_001",
    details_json='{"dim":"gold_dim_user","key":"client_id"}'
)

append_anomalies(
    df_orphan_card,
    anom_type="ORPHAN_DIM_CARD",
    severity="ERROR",
    rule_id="GOLD_TXN_FK_002",
    details_json='{"dim":"gold_dim_card","key":"card_id"}'
)

append_anomalies(
    df_orphan_mcc,
    anom_type="ORPHAN_DIM_MCC",
    severity="WARN",
    rule_id="GOLD_TXN_FK_003",
    details_json='{"dim":"gold_dim_mcc","key":"mcc_code"}'
)

append_anomalies(
    df_orphan_date,
    anom_type="ORPHAN_DIM_DATE",
    severity="ERROR",
    rule_id="GOLD_TXN_FK_004",
    details_json='{"dim":"gold_dim_date","key":"txn_date"}'
)



StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 11, Finished, Available, Finished)

Anomalies appended to gold_anomaly_event: ORPHAN_DIM_DATE (ERROR)


In [10]:
#---------------------------------------
# Persist AGG Anomaly KPIs
#---------------------------------------

KPI_TABLE = "gold_anomaly_kpi"

def append_anomaly_kpi(df: DataFrame, anom_type: str, severity: str, rule_id: str):
    cnt = df.count()
    if cnt == 0:
        return
    spark.createDataFrame(
        [(ANOM_DOMAIN, anom_type, severity, rule_id, int(cnt), gold_run_id)],
        ["anom_domain","anom_type","severity","rule_id","anomaly_count","gold_run_id"]
    ).withColumn("gold_load_ts", F.current_timestamp()) \
     .write.mode("append").format("delta").saveAsTable(KPI_TABLE)

append_anomaly_kpi(df_orphan_user, "ORPHAN_DIM_USER", "ERROR", "GOLD_TXN_FK_001")
append_anomaly_kpi(df_orphan_card, "ORPHAN_DIM_CARD", "ERROR", "GOLD_TXN_FK_002")
append_anomaly_kpi(df_orphan_mcc,  "ORPHAN_DIM_MCC",  "WARN",  "GOLD_TXN_FK_003")
append_anomaly_kpi(df_orphan_date, "ORPHAN_DIM_DATE", "ERROR", "GOLD_TXN_FK_004")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 12, Finished, Available, Finished)

In [11]:
#---------------------------------------
# Filter conformed transactions
#---------------------------------------

df_conformed = (
    df_dedup
    .join(df_users, on="client_id", how="left_semi")
    .join(df_cards, on="card_id", how="left_semi")
    .join(df_mcc,   on="mcc_code", how="left_semi")
    .join(df_dates, on="txn_date", how="left_semi")
)

print(f"Conformed transactions count = {df_conformed.count()}")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 13, Finished, Available, Finished)

Conformed transactions count = 6734248


In [12]:
#--------------------------------------------
# Technical columns Gold + Write Partitionned
#--------------------------------------------

df_gold = (
    df_conformed
    .withColumn("gold_run_id", F.lit(gold_run_id))
    .withColumn("gold_load_ts", F.current_timestamp())
)

# Rebuild contrôlé : TRUNCATE + append
spark.sql(f"TRUNCATE TABLE {GOLD_TABLE}")

(
    df_gold
    .write
    .mode("append")
    .format("delta")
    .partitionBy("txn_month")
    .saveAsTable(GOLD_TABLE)
)

print(f"Loaded {GOLD_TABLE} successfully.")


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 14, Finished, Available, Finished)

Loaded gold_fact_transactions successfully.


In [13]:
#---------------------------------------
# Post Load Control
#---------------------------------------

spark.sql(f"""
SELECT
  txn_month,
  count(*) AS txn_cnt,
  sum(amount) AS total_amount,
  avg(amount) AS avg_amount,
  avg(CASE WHEN is_success THEN 1.0 ELSE 0.0 END) AS success_rate
FROM {GOLD_TABLE}
GROUP BY txn_month
ORDER BY txn_month
LIMIT 24
""").show(truncate=False)

spark.sql(f"SELECT * FROM {GOLD_TABLE} ORDER BY txn_ts DESC LIMIT 20").show(truncate=False)


StatementMeta(, 5f0c47b2-368c-43f2-8f3c-91c8ece1fdda, 15, Finished, Available, Finished)

+----------+-------+------------+----------+------------+
|txn_month |txn_cnt|total_amount|avg_amount|success_rate|
+----------+-------+------------+----------+------------+
|2015-01-01|116649 |4953472.30  |42.464764 |1.00000     |
|2015-02-01|104862 |4526582.18  |43.167040 |1.00000     |
|2015-03-01|117987 |5076515.69  |43.026060 |1.00000     |
|2015-04-01|113157 |4889663.36  |43.211320 |1.00000     |
|2015-05-01|117561 |5083713.60  |43.243198 |1.00000     |
|2015-06-01|114838 |4946303.00  |43.072006 |1.00000     |
|2015-07-01|118686 |5084102.70  |42.836583 |1.00000     |
|2015-08-01|118453 |5042418.58  |42.568939 |1.00000     |
|2015-09-01|114379 |4925569.11  |43.063579 |1.00000     |
|2015-10-01|117621 |5050182.67  |42.936063 |1.00000     |
|2015-11-01|114476 |4823851.42  |42.138539 |1.00000     |
|2015-12-01|119396 |5111632.82  |42.812429 |1.00000     |
|2016-01-01|117907 |5071138.30  |43.009646 |1.00000     |
|2016-02-01|106674 |4612980.36  |43.243718 |1.00000     |
|2016-03-01|11